# 🏏 IPL Team Performance Analysis Dashboard

## Overview
This notebook provides comprehensive analysis of IPL team and player performance, including:
- **Phase Analysis**: Compare run rates across Powerplay (1-6), Middle (7-15), and Death (16-20) overs
- **Matchup Analysis**: Analyze batter performance vs different bowler types
- **Interactive 3D Visualizations**: Using Three.js for dynamic 3D charts

---

**Data Source**: Cricsheet.org (Men's IPL ball-by-ball data)

## 1. Install and Import Required Libraries

In [2]:
# Install required packages (uncomment if running for the first time)
# !pip install pandas numpy ipywidgets -q

import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


## 2. Download and Load IPL Dataset

Download the latest IPL ball-by-ball data from Cricsheet.org

In [3]:
import pandas as pd
import glob
import os

# Path to your extracted data folder
path = "ipl_data" 

# Check if the data folder exists
if not os.path.exists(path):
    print(f"⚠️ Folder '{path}' not found. Creating sample data...")
    
    # Create sample data with all major IPL teams
    import numpy as np
    np.random.seed(42)
    
    teams = [
        'Chennai Super Kings',
        'Mumbai Indians',
        'Royal Challengers Bangalore',
        'Kolkata Knight Riders',
        'Delhi Capitals',
        'Rajasthan Royals',
        'Punjab Kings',
        'Gujarat Titans',
        'Lucknow Super Giants',
        'Sunrisers Hyderabad'
    ]
    
    batters = [
        'MS Dhoni', 'Ruturaj Gaikwad', 'Devon Conway',
        'Rohit Sharma', 'Ishan Kishan', 'Suryakumar Yadav',
        'Virat Kohli', 'Faf du Plessis', 'Glenn Maxwell',
        'Shreyas Iyer', 'Venkatesh Iyer', 'Andre Russell',
        'David Warner', 'Rishabh Pant', 'Prithvi Shaw',
        'Jos Buttler', 'Sanju Samson', 'Yashasvi Jaiswal',
        'Shikhar Dhawan', 'Jonny Bairstow', 'Liam Livingstone',
        'Hardik Pandya', 'Shubman Gill', 'Rashid Khan',
        'KL Rahul', 'Nicholas Pooran', 'Quinton de Kock',
    ]
    
    bowlers = [
        'Jasprit Bumrah', 'Trent Boult', 'Mohammed Siraj',
        'Ravindra Jadeja', 'Axar Patel', 'Kuldeep Yadav',
        'Rashid Khan', 'Yuzvendra Chahal', 'Ravichandran Ashwin',
        'Kagiso Rabada', 'Bhuvneshwar Kumar', 'Mohammed Shami'
    ]
    
    n_records = 12000
    
    final_df = pd.DataFrame({
        'match_id': np.repeat(range(1, 101), 120),
        'season': np.random.choice([2020, 2021, 2022, 2023, 2024], n_records),
        'batting_team': np.random.choice(teams, n_records),
        'bowling_team': np.random.choice(teams, n_records),
        'ball': np.tile(np.arange(0.1, 20.1, 0.1), n_records // 200 + 1)[:n_records],
        'batter': np.random.choice(batters, n_records),
        'bowler': np.random.choice(bowlers, n_records),
        'runs_off_bat': np.random.choice([0, 1, 2, 3, 4, 6], n_records, p=[0.35, 0.30, 0.15, 0.05, 0.10, 0.05]),
        'extras': np.random.choice([0, 1, 2], n_records, p=[0.85, 0.12, 0.03]),
        'wicket_type': np.random.choice([None, 'caught', 'bowled', 'lbw', 'run out', 'stumped'], 
                                       n_records, p=[0.92, 0.03, 0.02, 0.015, 0.01, 0.005])
    })
    
    # Ensure batting and bowling teams are different
    same_team = final_df['batting_team'] == final_df['bowling_team']
    while same_team.any():
        final_df.loc[same_team, 'bowling_team'] = np.random.choice(teams, same_team.sum())
        same_team = final_df['batting_team'] == final_df['bowling_team']
    
    print(f"✅ Sample data created with {len(final_df):,} records")
    print(f"🏏 Teams: {', '.join(teams)}")
else:
    # Get a list of ALL csv files
    all_files = glob.glob(os.path.join(path, "*.csv"))
    
    print(f"Found {len(all_files)} match files.")
    
    # Read all files with error handling
    print("⏳ Merging all matches... this may take a minute...")
    df_list = []
    skipped = 0
    
    for i, filename in enumerate(all_files):
        try:
            # Try reading with error handling for bad lines
            df = pd.read_csv(filename, index_col=None, header=0, on_bad_lines='skip', encoding='utf-8')
            if not df.empty:
                df_list.append(df)
            
            # Progress indicator every 100 files
            if (i + 1) % 100 == 0:
                print(f"  Processed {i + 1}/{len(all_files)} files...")
        except Exception as e:
            skipped += 1
            continue
    
    if not df_list:
        print("⚠️ No valid data files found. Please check your data folder.")
        raise ValueError("No data could be loaded")
    
    # Combine them into one big DataFrame
    final_df = pd.concat(df_list, axis=0, ignore_index=True)
    
    print("✅ Done!")
    print(f"Total Data Points: {len(final_df):,}")
    print(f"Files skipped due to errors: {skipped}")
    
    # Get unique teams, filtering out NaN values
    if 'batting_team' in final_df.columns:
        unique_teams = final_df['batting_team'].dropna().unique()
        sorted_teams = sorted([str(team) for team in unique_teams])
        print(f"Teams ({len(sorted_teams)} total): {sorted_teams[:10]}")
    
    # Optional: Save this big file so you don't have to do this again
    final_df.to_csv("all_ipl_matches.csv", index=False)
    print("💾 Saved as 'all_ipl_matches.csv'")

Found 2338 match files.
⏳ Merging all matches... this may take a minute...
  Processed 100/2338 files...
  Processed 200/2338 files...
  Processed 300/2338 files...
  Processed 400/2338 files...
  Processed 500/2338 files...
  Processed 600/2338 files...
  Processed 700/2338 files...
  Processed 800/2338 files...
  Processed 900/2338 files...
  Processed 1000/2338 files...
  Processed 1100/2338 files...
  Processed 1200/2338 files...
  Processed 1300/2338 files...
  Processed 1400/2338 files...
  Processed 1500/2338 files...
  Processed 1600/2338 files...
  Processed 1700/2338 files...
  Processed 1800/2338 files...
  Processed 1900/2338 files...
  Processed 2000/2338 files...
  Processed 2100/2338 files...
  Processed 2200/2338 files...
  Processed 2300/2338 files...
✅ Done!
Total Data Points: 301,495
Files skipped due to errors: 0
Teams (19 total): ['Chennai Super Kings', 'Deccan Chargers', 'Delhi Capitals', 'Delhi Daredevils', 'Gujarat Lions', 'Gujarat Titans', 'Kings XI Punjab', 'K

## 3. Data Cleaning and Preparation

Clean the data and create additional columns for analysis

In [4]:
def clean_data(df):
    """Clean and prepare the data"""
    df = df.copy()
    
    # First, let's see what columns we have
    print(f"Available columns: {list(df.columns)[:15]}...")
    
    # Handle different column naming conventions from Cricsheet
    # Map common variations to standard names
    column_mappings = {
        'runs_off_bat': ['runs_off_bat', 'batsman_runs', 'runs_scored'],
        'extras': ['extras', 'extra_runs'],
        'wicket_type': ['wicket_type', 'dismissal_kind', 'wicket_kind'],
        'batter': ['batter', 'batsman', 'striker'],
        'bowler': ['bowler', 'bowler_name'],
        'ball': ['ball', 'over_ball'],
        'batting_team': ['batting_team', 'team'],
        'bowling_team': ['bowling_team', 'opponent']
    }
    
    # Standardize column names
    for standard_name, variations in column_mappings.items():
        for var in variations:
            if var in df.columns and standard_name not in df.columns:
                df.rename(columns={var: standard_name}, inplace=True)
                break
    
    # Create over number from ball column
    if 'ball' in df.columns:
        # Fill NaN values before converting to int
        df['ball'] = pd.to_numeric(df['ball'], errors='coerce').fillna(0)
        df['over'] = df['ball'].astype(int) + 1
    elif 'over' in df.columns:
        # Over might already exist in some formats
        df['over'] = pd.to_numeric(df['over'], errors='coerce').fillna(1).astype(int)
    else:
        print("⚠️ Warning: No 'ball' or 'over' column found")
        df['over'] = 1  # Default value
    
    # Calculate total runs
    if 'runs_off_bat' in df.columns and 'extras' in df.columns:
        df['runs_off_bat'] = pd.to_numeric(df['runs_off_bat'], errors='coerce').fillna(0)
        df['extras'] = pd.to_numeric(df['extras'], errors='coerce').fillna(0)
        df['total_runs'] = df['runs_off_bat'] + df['extras']
    elif 'runs_off_bat' in df.columns:
        df['runs_off_bat'] = pd.to_numeric(df['runs_off_bat'], errors='coerce').fillna(0)
        df['total_runs'] = df['runs_off_bat']
    else:
        print("⚠️ Warning: No runs columns found")
        df['total_runs'] = 0
    
    # Create phase column
    df['phase'] = pd.cut(df['over'], 
                         bins=[0, 6, 15, 21],
                         labels=['Powerplay (1-6)', 'Middle (7-15)', 'Death (16-20)'])
    
    # Create wicket indicator
    if 'wicket_type' in df.columns:
        df['is_wicket'] = df['wicket_type'].notna().astype(int)
    else:
        df['is_wicket'] = 0
    
    # Identify bowler type with more categories
    if 'bowler' in df.columns:
        def get_bowler_type(bowler_name):
            if pd.isna(bowler_name): return 'Unknown'
            name = str(bowler_name).lower()
            
            # Define keywords for different bowler types
            
            # Left Arm Pace
            left_arm_pacers = [
                'boult', 'arshdeep', 'natarajan', 'mustafizur', 'curran', 'starc', 'afridi', 
                'khaleel', 'jansen', 'behrendorff', 'yash dayal', 'mohsin khan', 'kotian', 
                'nandre burger', 'mcclenaghan', 'faulkner', 'johnson', 'zaheer', 'nehra', 
                'irfan pathan', 'r p singh', 'unadkat', 'cottrell', 'mills', 'udana', 'sakariya',
                'left-arm fast', 'left-arm medium'
            ]
            
            # Left Arm Wrist Spin (Chinaman)
            left_arm_wrist = ['kuldeep yadav', 'noor ahmad', 'tabraiz shamsi', 'chinaman']
            
            # Left Arm Orthodox
            left_arm_orthodox = [
                'jadeja', 'axar', 'krunal', 'shahbaz', 'santner', 'shakib', 'sai kishore', 
                'harpreet brar', 'abhishek sharma', 'nadeem', 'imad wasim', 'left-arm orthodox'
            ]
            
            # Right Arm Leg Spin
            right_arm_leg = [
                'rashid', 'chahal', 'bishnoi', 'hasaranga', 'zampa', 'mishra', 'rahul chahar', 
                'markande', 'chawla', 'karn sharma', 'gopal', 'tahir', 'badree', 'murugan ashwin',
                'legbreak', 'leg spin'
            ]
            
            # Right Arm Off Spin
            right_arm_off = [
                'ashwin', 'narine', 'chakravarthy', 'theekshana', 'livingstone', 'maxwell', 
                'moeen', 'gowtham', 'sundar', 'harbhajan', 'nabi', 'rana', 'hooda', 'parag',
                'off break', 'off spin'
            ]
            
            if any(p in name for p in left_arm_pacers):
                return 'Left-Arm Pace'
            elif any(p in name for p in left_arm_wrist):
                return 'Left-Arm Wrist Spin'
            elif any(p in name for p in left_arm_orthodox):
                return 'Left-Arm Orthodox'
            elif any(p in name for p in right_arm_leg):
                return 'Right-Arm Leg Spin'
            elif any(p in name for p in right_arm_off):
                return 'Right-Arm Off Spin'
            else:
                # Default assumption for others (mostly Right-Arm Pace)
                return 'Right-Arm Pace'

        df['bowler_type'] = df['bowler'].apply(get_bowler_type)
    else:
        df['bowler_type'] = 'Unknown'
    
    # Ensure batting_team and bowling_team don't have NaN
    if 'batting_team' in df.columns:
        df['batting_team'] = df['batting_team'].fillna('Unknown')
    if 'bowling_team' in df.columns:
        df['bowling_team'] = df['bowling_team'].fillna('Unknown')
    if 'batter' in df.columns:
        df['batter'] = df['batter'].fillna('Unknown')
    if 'bowler' in df.columns:
        df['bowler'] = df['bowler'].fillna('Unknown')
    
    return df

# Clean the data
df = clean_data(final_df)
print("\n✅ Data cleaned and prepared!")
print(f"📊 Dataset shape: {df.shape}")
print(f"🔧 New columns added: phase, total_runs, is_wicket, bowler_type")
print(f"📋 Sample teams: {df['batting_team'].unique()[:5]}")
print(f"🎯 Bowler Types found: {df['bowler_type'].unique()}")

Available columns: ['match_id', 'season', 'start_date', 'venue', 'innings', 'ball', 'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler', 'runs_off_bat', 'extras', 'wides', 'noballs']...

✅ Data cleaned and prepared!
📊 Dataset shape: (301495, 30)
🔧 New columns added: phase, total_runs, is_wicket, bowler_type
📋 Sample teams: ['Sunrisers Hyderabad' 'Royal Challengers Bangalore' 'Unknown'
 'Mumbai Indians' 'Rising Pune Supergiant']
🎯 Bowler Types found: ['Left-Arm Pace' 'Right-Arm Pace' 'Right-Arm Leg Spin'
 'Right-Arm Off Spin' 'Unknown' 'Left-Arm Wrist Spin' 'Left-Arm Orthodox']


## 4. Analysis Functions

Define functions for phase analysis, matchup analysis, and player statistics

In [5]:
def generate_stumps_view_data(df, team=None, phase=None):
    """Generate stumps view (behind bowler) data"""
    import numpy as np
    
    filtered_df = df.copy()
    if team:
        filtered_df = filtered_df[filtered_df['batting_team'] == team]
    if phase:
        filtered_df = filtered_df[filtered_df['phase'] == phase]
    
    if len(filtered_df) > 400:
        filtered_df = filtered_df.sample(400, random_state=42)
    
    np.random.seed(42)
    stumps_data = []
    
    for idx, row in filtered_df.iterrows():
        runs = row.get('runs_off_bat', 0)
        is_wicket = row.get('is_wicket', 0)
        
        # X: horizontal position (-2 to 2, left to right)
        # Y: vertical position (0 to 3, ground to top)
        
        if is_wicket:
            # Wickets - good line and length
            x = np.random.normal(0, 0.5)
            y = np.random.normal(1.5, 0.4)
            color = 'red'
            size = 6
        elif runs >= 6:
            # Sixes - shorter or fuller
            x = np.random.normal(0, 0.7)
            y = np.random.choice([np.random.normal(2.2, 0.3), np.random.normal(0.8, 0.3)])
            color = 'purple'
            size = 7
        elif runs == 4:
            # Fours - width variation
            x = np.random.normal(0, 0.9)
            y = np.random.normal(1.5, 0.5)
            color = 'green'
            size = 5
        elif runs in [1, 2, 3]:
            # Singles - good areas
            x = np.random.normal(0, 0.6)
            y = np.random.normal(1.5, 0.4)
            color = 'blue'
            size = 3
        else:
            # Dots - tight
            x = np.random.normal(0, 0.4)
            y = np.random.normal(1.5, 0.3)
            color = 'gray'
            size = 2
        
        # Clamp values
        x = max(-2.5, min(2.5, x))
        y = max(0.2, min(2.8, y))
        
        stumps_data.append({
            'x': float(x),
            'y': float(y),
            'runs': int(runs),
            'wicket': int(is_wicket),
            'color': color,
            'size': size,
            'batter': str(row.get('batter', 'Unknown')),
            'bowler': str(row.get('bowler', 'Unknown'))
        })
    
    return stumps_data

def generate_pitch_map_data_complete(df, team=None, bowler_type=None, phase=None):
    """Generate complete pitch map data with ball positions"""
    import numpy as np
    
    # Filter data
    filtered_df = df.copy()
    if team:
        filtered_df = filtered_df[filtered_df['batting_team'] == team]
    if bowler_type:
        filtered_df = filtered_df[filtered_df['bowler_type'] == bowler_type]
    if phase:
        filtered_df = filtered_df[filtered_df['phase'] == phase]
    
    # Sample data if too large (for performance)
    if len(filtered_df) > 500:
        filtered_df = filtered_df.sample(500, random_state=42)
    
    # Generate synthetic pitch positions
    np.random.seed(42)
    
    pitch_data = []
    for idx, row in filtered_df.iterrows():
        # Simulate pitch position based on outcome
        # X: -1 to 1 (left to right from bowler's perspective)
        # Y: 0 to 22 (pitch length in yards, 0 = bowler end, 22 = batter end)
        
        runs = row.get('runs_off_bat', 0)
        is_wicket = row.get('is_wicket', 0)
        
        # Good length balls (Y: 6-10 yards)
        # Short balls (Y: 0-6 yards)  
        # Full balls (Y: 10-16 yards)
        # Very full/yorkers (Y: 16-22 yards)
        
        if is_wicket:
            # Wicket balls tend to be good length, on or around off stump
            y = np.random.normal(8, 2)
            x = np.random.normal(0.3, 0.4)  # Around off stump
            color = 'red'
            size = 6
        elif runs >= 6:
            # Sixes - often short or very full
            y = np.random.choice([np.random.normal(4, 2), np.random.normal(18, 2)])
            x = np.random.normal(0, 0.6)
            color = 'purple'
            size = 7
        elif runs == 4:
            # Fours - various lengths
            y = np.random.normal(10, 4)
            x = np.random.normal(0, 0.7)
            color = 'green'
            size = 5
        elif runs in [1, 2, 3]:
            # Singles/doubles - good length
            y = np.random.normal(9, 3)
            x = np.random.normal(0, 0.5)
            color = 'blue'
            size = 3
        else:
            # Dot balls - good line and length
            y = np.random.normal(8, 2.5)
            x = np.random.normal(0.2, 0.4)
            color = 'gray'
            size = 2
        
        # Clamp values to pitch boundaries
        x = max(-1.2, min(1.2, x))
        y = max(0, min(22, y))
        
        pitch_data.append({
            'x': float(x),
            'y': float(y),
            'runs': int(runs),
            'wicket': int(is_wicket),
            'color': color,
            'size': size,
            'batter': str(row.get('batter', 'Unknown')),
            'bowler': str(row.get('bowler', 'Unknown'))
        })
    
    return pitch_data

print("✅ Analysis functions defined successfully!")

✅ Analysis functions defined successfully!


## 5. Visualization Functions

Create interactive Three.js 3D visualizations for analysis

In [6]:
import pandas as pd
import json
import uuid
from IPython.display import HTML, display

# Helper to ensure Three.js is loaded
def load_threejs():
    return HTML("""
    <script src="https://cdnjs.cloudflare.com/ajax/libs/three.js/r128/three.min.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/three@0.128.0/examples/js/controls/OrbitControls.js"></script>
    """)

display(load_threejs())

def create_threejs_chart(div_id, data, chart_type, title, width=600, height=400):
    """Generates HTML/JS for Three.js 3D charts"""
    
    style = """
    <style>
        .chart-container { position: relative; margin: 20px 0; }
        .chart-title { text-align: center; font-size: 18px; font-weight: bold; margin-bottom: 10px; }
        .chart-canvas { border: 1px solid #ddd; border-radius: 8px; }
        .tooltip-3d { position: absolute; background: rgba(0,0,0,0.8); color: white; padding: 8px 12px; 
                      border-radius: 4px; font-size: 12px; pointer-events: none; display: none; z-index: 1000; }
    </style>
    """
    
    script = ""
    
    if chart_type == 'grouped_bar_3d':
        # 3D Grouped Bar Chart
        script = f"""
        const container = document.getElementById('{div_id}');
        const tooltip = document.getElementById('{div_id}-tooltip');
        
        const scene = new THREE.Scene();
        scene.background = new THREE.Color(0xf8f9fa);
        
        const camera = new THREE.PerspectiveCamera(60, {width}/{height}, 0.1, 1000);
        camera.position.set(18, 18, 18);
        
        const renderer = new THREE.WebGLRenderer({{ antialias: true }});
        renderer.setSize({width}, {height});
        renderer.setPixelRatio(window.devicePixelRatio);
        renderer.shadowMap.enabled = true;
        renderer.shadowMap.type = THREE.PCFSoftShadowMap;
        container.appendChild(renderer.domElement);
        
        const controls = new THREE.OrbitControls(camera, renderer.domElement);
        controls.enableDamping = true;
        controls.dampingFactor = 0.08;
        controls.minDistance = 10;
        controls.maxDistance = 40;
        controls.maxPolarAngle = Math.PI / 2.2;
        
        // Lighting
        const ambientLight = new THREE.AmbientLight(0xffffff, 0.6);
        scene.add(ambientLight);
        const directionalLight = new THREE.DirectionalLight(0xffffff, 0.8);
        directionalLight.position.set(10, 20, 10);
        scene.add(directionalLight);
        
        // Data processing
        const categories = {json.dumps([item['category'] for item in data])};
        const teamNames = {json.dumps([val['label'] for val in data[0]['values']])};
        const colors = [0xFDB913, 0x004BA0]; // Team colors
        
        // Find max value for scaling
        let maxValue = 0;
        data.forEach(cat => {{
            cat.values.forEach(val => {{
                if (val.value > maxValue) maxValue = val.value;
            }});
        }});
        
        const barWidth = 1.5;
        const barDepth = 1.5;
        const spacing = 5;
        const groupSpacing = 2;
        
        // Create bars
        data.forEach((category, catIndex) => {{
            category.values.forEach((val, teamIndex) => {{
                const height = (val.value / maxValue) * 10;
                const geometry = new THREE.BoxGeometry(barWidth, height, barDepth);
                const material = new THREE.MeshPhongMaterial({{ 
                    color: colors[teamIndex],
                    shininess: 100
                }});
                const bar = new THREE.Mesh(geometry, material);
                
                const xPos = catIndex * spacing;
                const zPos = teamIndex * groupSpacing - 1;
                bar.position.set(xPos, height/2, zPos);
                
                bar.userData = {{
                    team: val.label,
                    phase: category.category,
                    value: val.value.toFixed(2),
                    balls: val.balls || 0,
                    wickets: val.wickets || 0
                }};
                
                scene.add(bar);
            }});
        }});
        
        // Grid and axes
        const gridHelper = new THREE.GridHelper(20, 20, 0x888888, 0xcccccc);
        scene.add(gridHelper);
        
        // Raycaster for interaction
        const raycaster = new THREE.Raycaster();
        const mouse = new THREE.Vector2();
        
        renderer.domElement.addEventListener('mousemove', (event) => {{
            const rect = renderer.domElement.getBoundingClientRect();
            mouse.x = ((event.clientX - rect.left) / rect.width) * 2 - 1;
            mouse.y = -((event.clientY - rect.top) / rect.height) * 2 + 1;
            
            raycaster.setFromCamera(mouse, camera);
            const intersects = raycaster.intersectObjects(scene.children);
            
            if (intersects.length > 0 && intersects[0].object.userData.team) {{
                const data = intersects[0].object.userData;
                tooltip.innerHTML = `
                    <strong>${{data.team}}</strong><br>
                    ${{data.phase}}<br>
                    Run Rate: ${{data.value}}<br>
                    Balls: ${{data.balls}}, Wickets: ${{data.wickets}}
                `;
                tooltip.style.display = 'block';
                tooltip.style.left = event.clientX + 10 + 'px';
                tooltip.style.top = event.clientY + 10 + 'px';
            }} else {{
                tooltip.style.display = 'none';
            }}
        }});
        
        function animate() {{
            requestAnimationFrame(animate);
            controls.update();
            renderer.render(scene, camera);
        }}
        animate();
        """
    
    elif chart_type == 'bar_3d':
        # 3D Bar Chart
        script = f"""
        const container = document.getElementById('{div_id}');
        const tooltip = document.getElementById('{div_id}-tooltip');
        
        const scene = new THREE.Scene();
        scene.background = new THREE.Color(0xf8f9fa);
        
        const camera = new THREE.PerspectiveCamera(60, {width}/{height}, 0.1, 1000);
        camera.position.set(10, 10, 15);
        
        const renderer = new THREE.WebGLRenderer({{ antialias: true }});
        renderer.setSize({width}, {height});
        container.appendChild(renderer.domElement);
        
        const controls = new THREE.OrbitControls(camera, renderer.domElement);
        controls.enableDamping = true;
        
        const ambientLight = new THREE.AmbientLight(0xffffff, 0.6);
        scene.add(ambientLight);
        const directionalLight = new THREE.DirectionalLight(0xffffff, 0.8);
        directionalLight.position.set(10, 20, 10);
        scene.add(directionalLight);
        
        const maxValue = Math.max(...data.map(d => d.value));
        const barWidth = 1.5;
        const spacing = 3;
        
        data.forEach((item, index) => {{
            const height = (item.value / maxValue) * 10;
            const hue = (index / data.length) * 0.7;
            
            const geometry = new THREE.BoxGeometry(barWidth, height, barWidth);
            const material = new THREE.MeshPhongMaterial({{ 
                color: new THREE.Color().setHSL(hue, 0.7, 0.5),
                shininess: 100
            }});
            const bar = new THREE.Mesh(geometry, material);
            
            bar.position.set((index - data.length/2) * spacing, height/2, 0);
            bar.userData = {{
                player: item.label,
                strikeRate: item.value.toFixed(2),
                balls: item.balls || 0,
                runs: item.runs || 0,
                dismissals: item.dismissals || 0
            }};
            
            scene.add(bar);
        }});
        
        const gridHelper = new THREE.GridHelper(20, 20, 0x888888, 0xcccccc);
        scene.add(gridHelper);
        
        const raycaster = new THREE.Raycaster();
        const mouse = new THREE.Vector2();
        
        renderer.domElement.addEventListener('mousemove', (event) => {{
            const rect = renderer.domElement.getBoundingClientRect();
            mouse.x = ((event.clientX - rect.left) / rect.width) * 2 - 1;
            mouse.y = -((event.clientY - rect.top) / rect.height) * 2 + 1;
            
            raycaster.setFromCamera(mouse, camera);
            const intersects = raycaster.intersectObjects(scene.children);
            
            if (intersects.length > 0 && intersects[0].object.userData.player) {{
                const data = intersects[0].object.userData;
                tooltip.innerHTML = `
                    <strong>${{data.player}}</strong><br>
                    Strike Rate: ${{data.strikeRate}}<br>
                    Runs: ${{data.runs}}, Balls: ${{data.balls}}<br>
                    Dismissals: ${{data.dismissals}}
                `;
                tooltip.style.display = 'block';
                tooltip.style.left = event.clientX + 10 + 'px';
                tooltip.style.top = event.clientY + 10 + 'px';
            }} else {{
                tooltip.style.display = 'none';
            }}
        }});
        
        function animate() {{
            requestAnimationFrame(animate);
            controls.update();
            renderer.render(scene, camera);
        }}
        animate();
        """
    
    elif chart_type == 'pie_3d':
        # 3D Pie Chart (Donut)
        script = f"""
        const container = document.getElementById('{div_id}');
        const tooltip = document.getElementById('{div_id}-tooltip');
        
        const scene = new THREE.Scene();
        scene.background = new THREE.Color(0xf8f9fa);
        
        const camera = new THREE.PerspectiveCamera(60, {width}/{height}, 0.1, 1000);
        camera.position.set(0, 8, 12);
        
        const renderer = new THREE.WebGLRenderer({{ antialias: true }});
        renderer.setSize({width}, {height});
        container.appendChild(renderer.domElement);
        
        const controls = new THREE.OrbitControls(camera, renderer.domElement);
        controls.enableDamping = true;
        
        const ambientLight = new THREE.AmbientLight(0xffffff, 0.6);
        scene.add(ambientLight);
        const directionalLight = new THREE.DirectionalLight(0xffffff, 0.8);
        directionalLight.position.set(5, 10, 5);
        scene.add(directionalLight);
        
        const total = data.reduce((sum, item) => sum + item.value, 0);
        let currentAngle = 0;
        const innerRadius = 2;
        const outerRadius = 5;
        const depth = 1;
        
        data.forEach((item, index) => {{
            const angle = (item.value / total) * Math.PI * 2;
            const hue = (index / data.length);
            
            const shape = new THREE.Shape();
            const startAngle = currentAngle;
            const endAngle = currentAngle + angle;
            
            // Create donut segment
            shape.moveTo(outerRadius * Math.cos(startAngle), outerRadius * Math.sin(startAngle));
            shape.absarc(0, 0, outerRadius, startAngle, endAngle, false);
            shape.lineTo(innerRadius * Math.cos(endAngle), innerRadius * Math.sin(endAngle));
            shape.absarc(0, 0, innerRadius, endAngle, startAngle, true);
            
            const geometry = new THREE.ExtrudeGeometry(shape, {{
                depth: depth,
                bevelEnabled: true,
                bevelThickness: 0.1,
                bevelSize: 0.1,
                bevelSegments: 2
            }});
            
            const material = new THREE.MeshPhongMaterial({{ 
                color: new THREE.Color().setHSL(hue, 0.8, 0.6),
                shininess: 100
            }});
            
            const mesh = new THREE.Mesh(geometry, material);
            mesh.position.z = -depth/2;
            mesh.userData = {{
                label: item.label,
                value: item.value,
                percentage: ((item.value / total) * 100).toFixed(1)
            }};
            
            scene.add(mesh);
            currentAngle = endAngle;
        }});
        
        const raycaster = new THREE.Raycaster();
        const mouse = new THREE.Vector2();
        
        renderer.domElement.addEventListener('mousemove', (event) => {{
            const rect = renderer.domElement.getBoundingClientRect();
            mouse.x = ((event.clientX - rect.left) / rect.width) * 2 - 1;
            mouse.y = -((event.clientY - rect.top) / rect.height) * 2 + 1;
            
            raycaster.setFromCamera(mouse, camera);
            const intersects = raycaster.intersectObjects(scene.children);
            
            if (intersects.length > 0 && intersects[0].object.userData.label) {{
                const data = intersects[0].object.userData;
                tooltip.innerHTML = `
                    <strong>Runs: ${{data.label}}</strong><br>
                    Count: ${{data.value}}<br>
                    Percentage: ${{data.percentage}}%
                `;
                tooltip.style.display = 'block';
                tooltip.style.left = event.clientX + 10 + 'px';
                tooltip.style.top = event.clientY + 10 + 'px';
            }} else {{
                tooltip.style.display = 'none';
            }}
        }});
        
        function animate() {{
            requestAnimationFrame(animate);
            controls.update();
            renderer.render(scene, camera);
        }}
        animate();
        """
    
    html_content = f"""
    {style}
    <div class="chart-container">
        <div class="chart-title">{title}</div>
        <div id="{div_id}" class="chart-canvas"></div>
        <div id="{div_id}-tooltip" class="tooltip-3d"></div>
    </div>
    <script>
    (function() {{
        const data = {json.dumps(data)};
        {script}
    }})();
    </script>
    """
    return HTML(html_content)

def plot_phase_comparison(df, team1, team2):
    """3D Phase comparison chart"""
    t1_stats = calculate_run_rate_by_phase(df, team1)
    t2_stats = calculate_run_rate_by_phase(df, team2)
    
    phases = ['Powerplay (1-6)', 'Middle (7-15)', 'Death (16-20)']
    data = []
    
    for phase in phases:
        val1 = float(t1_stats[t1_stats['phase'] == phase]['run_rate'].values[0]) if not t1_stats[t1_stats['phase'] == phase].empty else 0.0
        val2 = float(t2_stats[t2_stats['phase'] == phase]['run_rate'].values[0]) if not t2_stats[t2_stats['phase'] == phase].empty else 0.0
        balls1 = int(t1_stats[t1_stats['phase'] == phase]['ball'].values[0]) if not t1_stats[t1_stats['phase'] == phase].empty else 0
        balls2 = int(t2_stats[t2_stats['phase'] == phase]['ball'].values[0]) if not t2_stats[t2_stats['phase'] == phase].empty else 0
        
        data.append({
            'category': str(phase),
            'values': [
                {'label': str(team1), 'value': val1, 'balls': balls1, 'wickets': 0},
                {'label': str(team2), 'value': val2, 'balls': balls2, 'wickets': 0}
            ]
        })
        
    return create_threejs_chart(f"chart_phase_{uuid.uuid4().hex[:8]}", data, 'grouped_bar_3d', f"Run Rate: {team1} vs {team2}", 700, 500)

def plot_matchup_heatmap(df, team, bowler_type):
    """3D Bar chart for player matchups"""
    batters = get_top_batters(df, team, n=5)['batter'].tolist()
    
    data = []
    for batter in batters:
        stats = calculate_player_matchup(df, batter, bowler_type)
        if stats:
            data.append({
                'label': str(batter),
                'value': float(stats['strike_rate']),
                'balls': int(stats['balls_faced']),
                'runs': int(stats['runs_scored']),
                'dismissals': int(stats['dismissals'])
            })
            
    if not data:
        return HTML(f"<p>No data for {team} vs {bowler_type}</p>")
        
    return create_threejs_chart(f"chart_matchup_{uuid.uuid4().hex[:8]}", data, 'bar_3d', f"{team} Batters vs {bowler_type}", 600, 450)

def plot_runs_distribution(df, team):
    """3D Pie chart for runs distribution"""
    team_data = df[df['batting_team'] == team]
    runs_dist = team_data['runs_off_bat'].value_counts().sort_index()
    
    data = [{'label': str(k), 'value': int(v)} for k, v in runs_dist.items()]
    
    return create_threejs_chart(f"chart_runs_{uuid.uuid4().hex[:8]}", data, 'pie_3d', f"Runs Distribution - {team}", 500, 400)

print("✅ Three.js 3D Visualization functions created!")

✅ Three.js 3D Visualization functions created!


In [7]:
def create_pitch_map(df, team, phase=None, bowler_type=None):
    """Create 3D pitch map visualization"""
    pitch_data = generate_pitch_map_data(df, team=team, phase=phase, bowler_type=bowler_type)
    
    if not pitch_data:
        return HTML("<p>No data available for pitch map</p>")
    
    div_id = f"pitch_map_{uuid.uuid4().hex[:8]}"
    data_json = json.dumps(pitch_data)
    
    title_parts = [f"{team} Pitch Map"]
    if phase:
        title_parts.append(f"- {phase}")
    if bowler_type:
        title_parts.append(f"vs {bowler_type}")
    title = " ".join(title_parts)
    
    html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/three.js/r128/three.min.js"></script>
        <script src="https://cdn.jsdelivr.net/npm/three@0.128.0/examples/js/controls/OrbitControls.js"></script>
        <style>
            body {{ margin: 0; padding: 20px; font-family: sans-serif; }}
            .pitch-container {{ position: relative; }}
            .pitch-title {{ text-align: center; font-size: 18px; font-weight: bold; margin-bottom: 10px; }}
            .pitch-legend {{ 
                position: absolute; 
                top: 60px; 
                right: 20px; 
                background: rgba(255,255,255,0.9); 
                padding: 12px; 
                border-radius: 6px;
                font-size: 12px;
                box-shadow: 0 2px 8px rgba(0,0,0,0.2);
            }}
            .legend-item {{ 
                display: flex; 
                align-items: center; 
                margin: 4px 0; 
            }}
            .legend-color {{ 
                width: 16px; 
                height: 16px; 
                border-radius: 50%; 
                margin-right: 8px; 
            }}
            .tooltip-pitch {{ 
                position: absolute; 
                background: rgba(0,0,0,0.85); 
                color: white; 
                padding: 8px 12px; 
                border-radius: 4px; 
                font-size: 11px; 
                pointer-events: none; 
                display: none; 
                z-index: 1000;
            }}
            #{div_id} {{ border: 1px solid #ddd; border-radius: 8px; }}
        </style>
    </head>
    <body>
        <div class="pitch-container">
            <div class="pitch-title">{title}</div>
            <div class="pitch-legend">
                <div style="font-weight: bold; margin-bottom: 6px;">Ball Outcome:</div>
                <div class="legend-item">
                    <div class="legend-color" style="background: gray;"></div>
                    <span>Dot Ball (0)</span>
                </div>
                <div class="legend-item">
                    <div class="legend-color" style="background: blue;"></div>
                    <span>1-3 Runs</span>
                </div>
                <div class="legend-item">
                    <div class="legend-color" style="background: green;"></div>
                    <span>Four (4)</span>
                </div>
                <div class="legend-item">
                    <div class="legend-color" style="background: purple;"></div>
                    <span>Six (6)</span>
                </div>
                <div class="legend-item">
                    <div class="legend-color" style="background: red;"></div>
                    <span>Wicket</span>
                </div>
            </div>
            <div id="{div_id}"></div>
            <div class="tooltip-pitch" id="{div_id}-tooltip"></div>
        </div>
        <script>
        (function() {{
            const pitchData = {data_json};
            const container = document.getElementById('{div_id}');
            const tooltip = document.getElementById('{div_id}-tooltip');
            
            const scene = new THREE.Scene();
            scene.background = new THREE.Color(0xf0f8ff);
            
            const camera = new THREE.PerspectiveCamera(50, 800/600, 0.1, 1000);
            camera.position.set(0, 25, 30);
            camera.lookAt(0, 0, 11);
            
            const renderer = new THREE.WebGLRenderer({{ antialias: true }});
            renderer.setSize(800, 600);
            container.appendChild(renderer.domElement);
            
            const controls = new THREE.OrbitControls(camera, renderer.domElement);
            controls.enableDamping = true;
            controls.target.set(0, 0, 11);
            
            // Lighting
            const ambientLight = new THREE.AmbientLight(0xffffff, 0.7);
            scene.add(ambientLight);
            const directionalLight = new THREE.DirectionalLight(0xffffff, 0.6);
            directionalLight.position.set(10, 20, 10);
            scene.add(directionalLight);
            
            // Create cricket pitch surface with real texture
            const pitchGeometry = new THREE.PlaneGeometry(2.4, 22);
            
            // Load cricket pitch texture
            const textureLoader = new THREE.TextureLoader();
            const pitchTexture = textureLoader.load(
                'https://images.unsplash.com/photo-1540747913346-19e32dc3e97e?w=800&h=1200&fit=crop',
                function(texture) {{
                    // Texture loaded successfully
                    texture.wrapS = THREE.RepeatWrapping;
                    texture.wrapT = THREE.RepeatWrapping;
                    texture.repeat.set(1, 1);
                }},
                undefined,
                function(error) {{
                    // Fallback to green color if texture fails to load
                    console.log('Texture load failed, using fallback color');
                }}
            );
            
            const pitchMaterial = new THREE.MeshPhongMaterial({{ 
                map: pitchTexture,
                color: 0x2d5016,  // Fallback color
                side: THREE.DoubleSide,
                shininess: 10
            }});
            
            const pitch = new THREE.Mesh(pitchGeometry, pitchMaterial);
            pitch.rotation.x = -Math.PI / 2;
            pitch.position.y = 0;
            pitch.position.z = 11;
            scene.add(pitch);
            
            // Create stumps at both ends
            const stumpMaterial = new THREE.MeshPhongMaterial({{ color: 0x8B4513 }});
            
            // Bowler's end stumps
            for (let i = -0.15; i <= 0.15; i += 0.15) {{
                const stumpGeometry = new THREE.CylinderGeometry(0.02, 0.02, 0.8);
                const stump = new THREE.Mesh(stumpGeometry, stumpMaterial);
                stump.position.set(i, 0.4, 0);
                scene.add(stump);
            }}
            
            // Batter's end stumps
            for (let i = -0.15; i <= 0.15; i += 0.15) {{
                const stumpGeometry = new THREE.CylinderGeometry(0.02, 0.02, 0.8);
                const stump = new THREE.Mesh(stumpGeometry, stumpMaterial);
                stump.position.set(i, 0.4, 22);
                scene.add(stump);
            }}
            
            // Create crease lines
            const creaseMaterial = new THREE.MeshBasicMaterial({{ color: 0xffffff }});
            
            // Bowling crease
            const bowlingCrease = new THREE.Mesh(
                new THREE.PlaneGeometry(2.64, 0.1),
                creaseMaterial
            );
            bowlingCrease.rotation.x = -Math.PI / 2;
            bowlingCrease.position.set(0, 0.01, 0);
            scene.add(bowlingCrease);
            
            // Batting crease
            const battingCrease = new THREE.Mesh(
                new THREE.PlaneGeometry(2.64, 0.1),
                creaseMaterial
            );
            battingCrease.rotation.x = -Math.PI / 2;
            battingCrease.position.set(0, 0.01, 22);
            scene.add(battingCrease);
            
            // Color mapping
            const colorMap = {{
                'red': 0xff0000,
                'purple': 0x9c27b0,
                'green': 0x00ff00,
                'blue': 0x2196f3,
                'gray': 0x808080
            }};
            
            // Create balls on pitch
            const ballMeshes = [];
            pitchData.forEach(ball => {{
                const radius = ball.size * 0.03;
                const geometry = new THREE.SphereGeometry(radius, 16, 16);
                const material = new THREE.MeshPhongMaterial({{ 
                    color: colorMap[ball.color],
                    shininess: 100,
                    emissive: colorMap[ball.color],
                    emissiveIntensity: 0.3
                }});
                const sphere = new THREE.Mesh(geometry, material);
                
                sphere.position.set(ball.x, radius, ball.y);
                sphere.userData = {{
                    batter: ball.batter,
                    bowler: ball.bowler,
                    runs: ball.runs,
                    wicket: ball.wicket,
                    position: `(x: ${{ball.x.toFixed(2)}}, y: ${{ball.y.toFixed(1)}})`
                }};
                
                scene.add(sphere);
                ballMeshes.push(sphere);
            }});
            
            // Raycaster for hover effects
            const raycaster = new THREE.Raycaster();
            const mouse = new THREE.Vector2();
            
            renderer.domElement.addEventListener('mousemove', (event) => {{
                const rect = renderer.domElement.getBoundingClientRect();
                mouse.x = ((event.clientX - rect.left) / rect.width) * 2 - 1;
                mouse.y = -((event.clientY - rect.top) / rect.height) * 2 + 1;
                
                raycaster.setFromCamera(mouse, camera);
                const intersects = raycaster.intersectObjects(ballMeshes);
                
                if (intersects.length > 0 && intersects[0].object.userData.batter) {{
                    const data = intersects[0].object.userData;
                    let outcomeText = data.wicket ? 'WICKET' : `${{data.runs}} run(s)`;
                    tooltip.innerHTML = `
                        <strong>Outcome: ${{outcomeText}}</strong><br>
                        Batter: ${{data.batter}}<br>
                        Bowler: ${{data.bowler}}<br>
                        Position: ${{data.position}}
                    `;
                    tooltip.style.display = 'block';
                    tooltip.style.left = event.clientX + 10 + 'px';
                    tooltip.style.top = event.clientY + 10 + 'px';
                }} else {{
                    tooltip.style.display = 'none';
                }}
            }});
            
            // Animation loop
            function animate() {{
                requestAnimationFrame(animate);
                controls.update();
                renderer.render(scene, camera);
            }}
            animate();
        }})();
        </script>
    </body>
    </html>
    """
    return HTML(html)

print("✅ Pitch map visualization function created!")

✅ Pitch map visualization function created!


In [15]:
def create_wagon_wheel(df, team, batter=None, phase=None):
    """Create wagon wheel (ground map) visualization"""
    wagon_data = generate_wagon_wheel_data(df, team=team, batter=batter, phase=phase)
    
    if not wagon_data:
        return HTML("<p>No data available for wagon wheel</p>")
    
    div_id = f"wagon_wheel_{uuid.uuid4().hex[:8]}"
    data_json = json.dumps(wagon_data)
    
    title_parts = [f"{team} Wagon Wheel"]
    if batter:
        title_parts.insert(1, f"- {batter}")
    if phase:
        title_parts.append(f"({phase})")
    title = " ".join(title_parts)
    
    html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/three.js/r128/three.min.js"></script>
        <script src="https://cdn.jsdelivr.net/npm/three@0.128.0/examples/js/controls/OrbitControls.js"></script>
        <style>
            body {{ margin: 0; padding: 20px; font-family: sans-serif; }}
            .wagon-container {{ position: relative; display: inline-block; }}
            .wagon-title {{ text-align: center; font-size: 18px; font-weight: bold; margin-bottom: 10px; }}
            .wagon-legend {{ 
                position: absolute; 
                top: 60px; 
                right: 20px; 
                background: rgba(255,255,255,0.95); 
                padding: 10px; 
                border-radius: 6px;
                font-size: 11px;
                box-shadow: 0 2px 8px rgba(0,0,0,0.2);
            }}
            .legend-item {{ display: flex; align-items: center; margin: 3px 0; }}
            .legend-color {{ width: 14px; height: 14px; border-radius: 50%; margin-right: 6px; }}
            #{div_id} {{ border: 1px solid #ddd; border-radius: 8px; }}
        </style>
    </head>
    <body>
        <div class="wagon-container">
            <div class="wagon-title">{title}</div>
            <div class="wagon-legend">
                <div style="font-weight: bold; margin-bottom: 4px;">Runs:</div>
                <div class="legend-item"><div class="legend-color" style="background: purple;"></div><span>Six (6)</span></div>
                <div class="legend-item"><div class="legend-color" style="background: green;"></div><span>Four (4)</span></div>
                <div class="legend-item"><div class="legend-color" style="background: blue;"></div><span>1-3 Runs</span></div>
            </div>
            <div id="{div_id}"></div>
        </div>
        <script>
        (function() {{
            const wagonData = {data_json};
            const scene = new THREE.Scene();
            scene.background = new THREE.Color(0x0a5f0a);
            
            const camera = new THREE.OrthographicCamera(-110, 110, 110, -110, 0.1, 1000);
            camera.position.set(0, 100, 0);
            camera.lookAt(0, 0, 0);
            
            const renderer = new THREE.WebGLRenderer({{ antialias: true }});
            renderer.setSize(600, 600);
            document.getElementById('{div_id}').appendChild(renderer.domElement);
            
            const ambientLight = new THREE.AmbientLight(0xffffff, 0.8);
            scene.add(ambientLight);
            
            // Create circular ground
            const groundGeometry = new THREE.CircleGeometry(100, 64);
            const groundMaterial = new THREE.MeshBasicMaterial({{ color: 0x0d7a0d, side: THREE.DoubleSide }});
            const ground = new THREE.Mesh(groundGeometry, groundMaterial);
            ground.rotation.x = -Math.PI / 2;
            scene.add(ground);
            
            // Create inner circle (30-yard circle)
            const innerCircleGeometry = new THREE.RingGeometry(29.5, 30.5, 64);
            const innerCircleMaterial = new THREE.MeshBasicMaterial({{ color: 0xffffff, side: THREE.DoubleSide }});
            const innerCircle = new THREE.Mesh(innerCircleGeometry, innerCircleMaterial);
            innerCircle.rotation.x = -Math.PI / 2;
            scene.add(innerCircle);
            
            // Create boundary circle
            const boundaryGeometry = new THREE.RingGeometry(99, 100, 64);
            const boundaryMaterial = new THREE.MeshBasicMaterial({{ color: 0xffffff, side: THREE.DoubleSide }});
            const boundary = new THREE.Mesh(boundaryGeometry, boundaryMaterial);
            boundary.rotation.x = -Math.PI / 2;
            scene.add(boundary);
            
            // Create pitch (brown rectangle)
            const pitchGeometry = new THREE.PlaneGeometry(3, 22);
            const pitchMaterial = new THREE.MeshBasicMaterial({{ color: 0x8B6F47, side: THREE.DoubleSide }});
            const pitch = new THREE.Mesh(pitchGeometry, pitchMaterial);
            pitch.rotation.x = -Math.PI / 2;
            scene.add(pitch);
            
            // Create stumps
            const stumpMaterial = new THREE.MeshBasicMaterial({{ color: 0xffffff }});
            const stumpGeometry = new THREE.BoxGeometry(0.5, 0.5, 0.5);
            const stump1 = new THREE.Mesh(stumpGeometry, stumpMaterial);
            stump1.position.set(0, 0, 0);
            stump1.rotation.x = -Math.PI / 2;
            scene.add(stump1);
            
            // Color mapping
            const colorMap = {{
                'purple': 0x9c27b0,
                'green': 0x00ff00,
                'blue': 0x2196f3
            }};
            
            // Create shots as lines and dots
            const ballMeshes = [];
            wagonData.forEach(shot => {{
                // Draw line from center to shot location
                const lineMaterial = new THREE.LineBasicMaterial({{ 
                    color: colorMap[shot.color],
                    linewidth: 2,
                    opacity: 0.6,
                    transparent: true
                }});
                const lineGeometry = new THREE.BufferGeometry().setFromPoints([
                    new THREE.Vector3(0, 0.1, 0),
                    new THREE.Vector3(shot.x, 0.1, shot.y)
                ]);
                const line = new THREE.Line(lineGeometry, lineMaterial);
                scene.add(line);
                
                // Create ball at endpoint
                const radius = shot.size * 0.15;
                const ballGeometry = new THREE.SphereGeometry(radius, 16, 16);
                const ballMaterial = new THREE.MeshBasicMaterial({{ color: colorMap[shot.color] }});
                const ball = new THREE.Mesh(ballGeometry, ballMaterial);
                ball.position.set(shot.x, radius, shot.y);
                ball.userData = {{
                    runs: shot.runs,
                    batter: shot.batter,
                    bowler: shot.bowler,
                    angle: shot.angle.toFixed(1),
                    distance: shot.distance.toFixed(1)
                }};
                scene.add(ball);
                ballMeshes.push(ball);
            }});
            
            // Mouse interaction
            const raycaster = new THREE.Raycaster();
            const mouse = new THREE.Vector2();
            let tooltip = null;
            
            renderer.domElement.addEventListener('mousemove', (event) => {{
                const rect = renderer.domElement.getBoundingClientRect();
                mouse.x = ((event.clientX - rect.left) / rect.width) * 2 - 1;
                mouse.y = -((event.clientY - rect.top) / rect.height) * 2 + 1;
                
                raycaster.setFromCamera(mouse, camera);
                const intersects = raycaster.intersectObjects(ballMeshes);
                
                if (intersects.length > 0) {{
                    const data = intersects[0].object.userData;
                    if (!tooltip) {{
                        tooltip = document.createElement('div');
                        tooltip.style.position = 'absolute';
                        tooltip.style.background = 'rgba(0,0,0,0.85)';
                        tooltip.style.color = 'white';
                        tooltip.style.padding = '8px 12px';
                        tooltip.style.borderRadius = '4px';
                        tooltip.style.fontSize = '11px';
                        tooltip.style.pointerEvents = 'none';
                        tooltip.style.zIndex = '1000';
                        document.body.appendChild(tooltip);
                    }}
                    tooltip.innerHTML = `
                        <strong>${{data.runs}} run(s)</strong><br>
                        Batter: ${{data.batter}}<br>
                        Bowler: ${{data.bowler}}<br>
                        Angle: ${{data.angle}}°<br>
                        Distance: ${{data.distance}}m
                    `;
                    tooltip.style.display = 'block';
                    tooltip.style.left = event.clientX + 10 + 'px';
                    tooltip.style.top = event.clientY + 10 + 'px';
                }} else if (tooltip) {{
                    tooltip.style.display = 'none';
                }}
            }});
            
            renderer.render(scene, camera);
        }})();
        </script>
    </body>
    </html>
    """
    return HTML(html)

def create_stumps_view(df, team, phase=None):
    """Create stumps view (behind bowler perspective) visualization"""
    stumps_data = generate_stumps_view_data(df, team=team, phase=phase)
    
    if not stumps_data:
        return HTML("<p>No data available for stumps view</p>")
    
    div_id = f"stumps_view_{uuid.uuid4().hex[:8]}"
    data_json = json.dumps(stumps_data)
    
    title_parts = [f"{team} - Stumps View"]
    if phase:
        title_parts.append(f"({phase})")
    title = " ".join(title_parts)
    
    html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/three.js/r128/three.min.js"></script>
        <style>
            body {{ margin: 0; padding: 20px; font-family: sans-serif; }}
            .stumps-container {{ position: relative; display: inline-block; }}
            .stumps-title {{ text-align: center; font-size: 18px; font-weight: bold; margin-bottom: 10px; }}
            .stumps-legend {{ 
                position: absolute; 
                top: 60px; 
                right: 20px; 
                background: rgba(255,255,255,0.95); 
                padding: 10px; 
                border-radius: 6px;
                font-size: 11px;
                box-shadow: 0 2px 8px rgba(0,0,0,0.2);
            }}
            .legend-item {{ display: flex; align-items: center; margin: 3px 0; }}
            .legend-color {{ width: 14px; height: 14px; border-radius: 50%; margin-right: 6px; }}
            #{div_id} {{ border: 1px solid #ddd; border-radius: 8px; }}
        </style>
    </head>
    <body>
        <div class="stumps-container">
            <div class="stumps-title">{title}</div>
            <div class="stumps-legend">
                <div style="font-weight: bold; margin-bottom: 4px;">Outcome:</div>
                <div class="legend-item"><div class="legend-color" style="background: gray;"></div><span>Dot (0)</span></div>
                <div class="legend-item"><div class="legend-color" style="background: blue;"></div><span>1-3 Runs</span></div>
                <div class="legend-item"><div class="legend-color" style="background: green;"></div><span>Four</span></div>
                <div class="legend-item"><div class="legend-color" style="background: purple;"></div><span>Six</span></div>
                <div class="legend-item"><div class="legend-color" style="background: red;"></div><span>Wicket</span></div>
            </div>
            <div id="{div_id}"></div>
        </div>
        <script>
        (function() {{
            const stumpsData = {data_json};
            const scene = new THREE.Scene();
            scene.background = new THREE.Color(0xf0f8ff);
            
            const aspect = 500 / 600;
            const frustumSize = 7;
            const camera = new THREE.OrthographicCamera(
                frustumSize * aspect / -2, frustumSize * aspect / 2,
                frustumSize / 2, frustumSize / -2,
                0.1, 1000
            );
            camera.position.set(0, 0, 15);
            camera.lookAt(0, 1.5, 0);
            camera.zoom = 1.2;
            camera.updateProjectionMatrix();
            
            const renderer = new THREE.WebGLRenderer({{ antialias: true }});
            renderer.setSize(500, 600);
            renderer.setPixelRatio(window.devicePixelRatio);
            document.getElementById('{div_id}').appendChild(renderer.domElement);
            
            const ambientLight = new THREE.AmbientLight(0xffffff, 0.7);
            scene.add(ambientLight);
            
            const mainLight = new THREE.DirectionalLight(0xffffff, 0.6);
            mainLight.position.set(5, 10, 10);
            scene.add(mainLight);
            
            const fillLight = new THREE.DirectionalLight(0xffffff, 0.3);
            fillLight.position.set(-5, 5, 5);
            scene.add(fillLight);
            
            // Create pitch area background
            const pitchGeometry = new THREE.PlaneGeometry(6, 4);
            const pitchMaterial = new THREE.MeshBasicMaterial({{ color: 0xd4a574, side: THREE.DoubleSide }});
            const pitch = new THREE.Mesh(pitchGeometry, pitchMaterial);
            pitch.position.set(0, 1.5, -1);
            scene.add(pitch);
            
            // Create stumps (3 vertical lines)
            const stumpMaterial = new THREE.MeshBasicMaterial({{ color: 0x8B4513 }});
            for (let i = -1; i <= 1; i++) {{
                const stumpGeometry = new THREE.CylinderGeometry(0.02, 0.02, 0.7, 8);
                const stump = new THREE.Mesh(stumpGeometry, stumpMaterial);
                stump.position.set(i * 0.12, 0.35, 0);
                scene.add(stump);
            }}
            
            // Create bails on top
            const bailMaterial = new THREE.MeshBasicMaterial({{ color: 0x8B4513 }});
            for (let i = -0.5; i <= 0.5; i++) {{
                const bailGeometry = new THREE.CylinderGeometry(0.01, 0.01, 0.12, 8);
                const bail = new THREE.Mesh(bailGeometry, bailMaterial);
                bail.rotation.z = Math.PI / 2;
                bail.position.set(i * 0.12, 0.7, 0);
                scene.add(bail);
            }}
            
            // Draw pitch lines
            const lineMaterial = new THREE.LineBasicMaterial({{ color: 0xffffff, linewidth: 2 }});
            const creaseGeometry = new THREE.BufferGeometry().setFromPoints([
                new THREE.Vector3(-0.6, 0, 0),
                new THREE.Vector3(0.6, 0, 0)
            ]);
            const crease = new THREE.Line(creaseGeometry, lineMaterial);
            scene.add(crease);
            
            // Color mapping
            const colorMap = {{
                'red': 0xff0000,
                'purple': 0x9c27b0,
                'green': 0x00ff00,
                'blue': 0x2196f3,
                'gray': 0x808080
            }};
            
            // Create balls
            const ballMeshes = [];
            stumpsData.forEach(ball => {{
                const radius = ball.size * 0.015;
                const ballGeometry = new THREE.SphereGeometry(radius, 16, 16);
                const ballMaterial = new THREE.MeshBasicMaterial({{ 
                    color: colorMap[ball.color],
                    opacity: 0.7,
                    transparent: true
                }});
                const sphere = new THREE.Mesh(ballGeometry, ballMaterial);
                sphere.position.set(ball.x, ball.y, 0.1);
                sphere.userData = {{
                    runs: ball.runs,
                    wicket: ball.wicket,
                    batter: ball.batter,
                    bowler: ball.bowler
                }};
                scene.add(sphere);
                ballMeshes.push(sphere);
            }});
            
            // Mouse interaction
            const raycaster = new THREE.Raycaster();
            const mouse = new THREE.Vector2();
            let tooltip = null;
            
            renderer.domElement.addEventListener('mousemove', (event) => {{
                const rect = renderer.domElement.getBoundingClientRect();
                mouse.x = ((event.clientX - rect.left) / rect.width) * 2 - 1;
                mouse.y = -((event.clientY - rect.top) / rect.height) * 2 + 1;
                
                raycaster.setFromCamera(mouse, camera);
                const intersects = raycaster.intersectObjects(ballMeshes);
                
                if (intersects.length > 0) {{
                    const data = intersects[0].object.userData;
                    if (!tooltip) {{
                        tooltip = document.createElement('div');
                        tooltip.style.position = 'absolute';
                        tooltip.style.background = 'rgba(0,0,0,0.85)';
                        tooltip.style.color = 'white';
                        tooltip.style.padding = '8px 12px';
                        tooltip.style.borderRadius = '4px';
                        tooltip.style.fontSize = '11px';
                        tooltip.style.pointerEvents = 'none';
                        tooltip.style.zIndex = '1000';
                        document.body.appendChild(tooltip);
                    }}
                    const wicketText = data.wicket ? ' - WICKET!' : '';
                    tooltip.innerHTML = `
                        <strong>${{data.runs}} run(s)${{wicketText}}</strong><br>
                        Batter: ${{data.batter}}<br>
                        Bowler: ${{data.bowler}}
                    `;
                    tooltip.style.display = 'block';
                    tooltip.style.left = event.clientX + 10 + 'px';
                    tooltip.style.top = event.clientY + 10 + 'px';
                }} else if (tooltip) {{
                    tooltip.style.display = 'none';
                }}
            }});
            
            renderer.render(scene, camera);
        }})();
        </script>
    </body>
    </html>
    """
    return HTML(html)

print("✅ New cricket visualization functions created (Wagon Wheel & Stumps View)!")

def create_advanced_pitch_viz(df, team, phase=None, bowler_type=None):
    """Create advanced pitch visualization with heat maps and density contours"""
    pitch_data = generate_pitch_map_data_complete(df, team=team, phase=phase, bowler_type=bowler_type)
    
    if not pitch_data:
        return HTML("<p>No data available for pitch visualization</p>")
    
    div_id = f"advanced_pitch_{uuid.uuid4().hex[:8]}"
    data_json = json.dumps(pitch_data)
    
    title_parts = [f"{team} Advanced Pitch Analysis"]
    if phase:
        title_parts.append(f"- {phase}")
    if bowler_type:
        title_parts.append(f"vs {bowler_type}")
    title = " ".join(title_parts)
    
    # Separate wickets and non-wickets
    wickets = [d for d in pitch_data if d['wicket'] == 1]
    hitting = [d for d in pitch_data if d['wicket'] == 0]
    
    html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <script src="https://cdn.plot.ly/plotly-2.26.0.min.js"></script>
        <style>
            body {{ margin: 0; padding: 20px; font-family: Arial, sans-serif; }}
            .viz-container {{ 
                display: grid; 
                grid-template-columns: repeat(4, 1fr); 
                gap: 15px; 
                margin: 20px 0;
            }}
            .viz-title {{ 
                text-align: center; 
                font-size: 20px; 
                font-weight: bold; 
                margin-bottom: 15px;
                color: #333;
            }}
            .viz-item {{ 
                border: 2px solid #ddd; 
                border-radius: 8px; 
                padding: 10px;
                background: white;
            }}
            .viz-subtitle {{ 
                text-align: center; 
                font-size: 14px; 
                font-weight: bold; 
                margin-bottom: 10px;
                color: #555;
            }}
            .legend {{ 
                position: fixed;
                top: 80px;
                right: 20px;
                background: rgba(255,255,255,0.95);
                padding: 12px;
                border-radius: 6px;
                box-shadow: 0 2px 10px rgba(0,0,0,0.2);
                font-size: 11px;
                z-index: 1000;
            }}
            .legend-item {{ display: flex; align-items: center; margin: 4px 0; }}
            .legend-color {{ width: 16px; height: 16px; border-radius: 50%; margin-right: 8px; }}
        </style>
    </head>
    <body>
        <div class="viz-title">{title}</div>
        <div class="legend">
            <div style="font-weight: bold; margin-bottom: 6px;">Ball Outcomes:</div>
            <div class="legend-item">
                <div class="legend-color" style="background: #ef5350;"></div>
                <span>Wickets</span>
            </div>
            <div class="legend-item">
                <div class="legend-color" style="background: #2196f3;"></div>
                <span>Not Hitting Wicket</span>
            </div>
            <div class="legend-item">
                <div class="legend-color" style="background: #ff9800;"></div>
                <span>High Density</span>
            </div>
        </div>
        
        <div class="viz-container">
            <div class="viz-item">
                <div class="viz-subtitle">Wickets - Ball Position</div>
                <div id="plot1"></div>
            </div>
            <div class="viz-item">
                <div class="viz-subtitle">Hitting - Ball Position</div>
                <div id="plot2"></div>
            </div>
            <div class="viz-item">
                <div class="viz-subtitle">Density Heat Map</div>
                <div id="plot3"></div>
            </div>
            <div class="viz-item">
                <div class="viz-subtitle">All Deliveries</div>
                <div id="plot4"></div>
            </div>
        </div>
        
        <script>
        (function() {{
            const allData = {data_json};
            const wickets = allData.filter(d => d.wicket === 1);
            const hitting = allData.filter(d => d.wicket === 0);
            
            // Common layout settings
            const commonLayout = {{
                width: 280,
                height: 400,
                margin: {{ t: 10, r: 10, b: 30, l: 30 }},
                xaxis: {{ 
                    range: [-1.5, 1.5], 
                    showgrid: true, 
                    gridcolor: 'white',
                    gridwidth: 2,
                    zeroline: false,
                    title: ''
                }},
                yaxis: {{ 
                    range: [0, 22], 
                    showgrid: true,
                    gridcolor: 'white',
                    gridwidth: 2,
                    zeroline: false,
                    title: ''
                }},
                plot_bgcolor: '#0a5f0a',
                paper_bgcolor: 'white',
                showlegend: false
            }};
            
            // Plot 1: Wickets scatter
            const wicketsTrace = {{
                x: wickets.map(d => d.x),
                y: wickets.map(d => d.y),
                mode: 'markers',
                type: 'scatter',
                marker: {{
                    size: 8,
                    color: '#ef5350',
                    opacity: 0.7,
                    line: {{ width: 1, color: 'white' }}
                }},
                hovertemplate: '<b>Wicket</b><br>Batter: %{{text}}<extra></extra>',
                text: wickets.map(d => d.batter)
            }};
            
            // Add pitch markings to plot 1
            const stumpLine1 = {{
                x: [-0.6, 0.6],
                y: [0, 0],
                mode: 'lines',
                line: {{ color: 'white', width: 3 }},
                hoverinfo: 'skip'
            }};
            
            const stumpLine2 = {{
                x: [-0.6, 0.6],
                y: [22, 22],
                mode: 'lines',
                line: {{ color: 'white', width: 3 }},
                hoverinfo: 'skip'
            }};
            
            Plotly.newPlot('plot1', [wicketsTrace, stumpLine1, stumpLine2], commonLayout, {{displayModeBar: false}});
            
            // Plot 2: Hitting scatter
            const hittingTrace = {{
                x: hitting.map(d => d.x),
                y: hitting.map(d => d.y),
                mode: 'markers',
                type: 'scatter',
                marker: {{
                    size: 6,
                    color: '#2196f3',
                    opacity: 0.6,
                    line: {{ width: 0.5, color: 'white' }}
                }},
                hovertemplate: '<b>%{{text}} run(s)</b><extra></extra>',
                text: hitting.map(d => d.runs)
            }};
            
            Plotly.newPlot('plot2', [hittingTrace, stumpLine1, stumpLine2], commonLayout, {{displayModeBar: false}});
            
            // Plot 3: Density heat map (2D histogram)
            const heatmapTrace = {{
                x: allData.map(d => d.x),
                y: allData.map(d => d.y),
                type: 'histogram2dcontour',
                colorscale: [
                    [0, '#0a5f0a'],
                    [0.2, '#1b7a1b'],
                    [0.4, '#ffeb3b'],
                    [0.6, '#ff9800'],
                    [0.8, '#ff5722'],
                    [1, '#b71c1c']
                ],
                showscale: true,
                colorbar: {{
                    len: 0.7,
                    thickness: 10,
                    x: 1.02
                }},
                contours: {{
                    coloring: 'heatmap',
                    showlabels: true
                }},
                hoverinfo: 'skip'
            }};
            
            Plotly.newPlot('plot3', [heatmapTrace, stumpLine1, stumpLine2], commonLayout, {{displayModeBar: false}});
            
            // Plot 4: Combined view with color coding
            const wicketsCombined = {{
                x: wickets.map(d => d.x),
                y: wickets.map(d => d.y),
                mode: 'markers',
                type: 'scatter',
                name: 'Wickets',
                marker: {{
                    size: 8,
                    color: '#ef5350',
                    opacity: 0.8
                }},
                hovertemplate: '<b>WICKET</b><br>%{{text}}<extra></extra>',
                text: wickets.map(d => d.batter)
            }};
            
            const hittingCombined = {{
                x: hitting.map(d => d.x),
                y: hitting.map(d => d.y),
                mode: 'markers',
                type: 'scatter',
                name: 'Hitting',
                marker: {{
                    size: 6,
                    color: '#ff9800',
                    opacity: 0.5
                }},
                hovertemplate: '<b>%{{text}} runs</b><extra></extra>',
                text: hitting.map(d => d.runs)
            }};
            
            const layout4 = {{
                ...commonLayout,
                showlegend: false
            }};
            
            Plotly.newPlot('plot4', [hittingCombined, wicketsCombined, stumpLine1, stumpLine2], layout4, {{displayModeBar: false}});

        }})();
        </script>
    </body>
    </html>
    """
    return HTML(html)

print("✅ Advanced pitch visualization with heat maps created!")

✅ New cricket visualization functions created (Wagon Wheel & Stumps View)!
✅ Advanced pitch visualization with heat maps created!


In [16]:
def get_player_statistics(df, team, phase=None):
    """Get comprehensive player statistics"""
    team_data = df[df['batting_team'] == team].copy()
    
    if phase:
        team_data = team_data[team_data['phase'] == phase]
    
    # Batter statistics
    batter_stats = team_data.groupby('batter').agg({
        'runs_off_bat': 'sum',
        'ball': 'count',
        'is_wicket': 'sum'
    }).reset_index()
    
    batter_stats = batter_stats[batter_stats['ball'] >= 30]  # Minimum 30 balls
    batter_stats['strike_rate'] = (batter_stats['runs_off_bat'] / batter_stats['ball'] * 100).round(2)
    batter_stats['average'] = (batter_stats['runs_off_bat'] / batter_stats['is_wicket'].replace(0, 1)).round(2)
    
    # Calculate 4s and 6s
    fours_sixes = team_data[team_data['runs_off_bat'].isin([4, 6])].groupby(['batter', 'runs_off_bat']).size().unstack(fill_value=0)
    if 4 in fours_sixes.columns:
        batter_stats = batter_stats.merge(fours_sixes[[4]].rename(columns={4: 'fours'}), left_on='batter', right_index=True, how='left')
    else:
        batter_stats['fours'] = 0
    if 6 in fours_sixes.columns:
        batter_stats = batter_stats.merge(fours_sixes[[6]].rename(columns={6: 'sixes'}), left_on='batter', right_index=True, how='left')
    else:
        batter_stats['sixes'] = 0
    
    batter_stats['fours'] = batter_stats['fours'].fillna(0).astype(int)
    batter_stats['sixes'] = batter_stats['sixes'].fillna(0).astype(int)
    
    # Calculate highest score per player (per innings)
    innings_scores = team_data.groupby(['batter', 'match_id'])['runs_off_bat'].sum().reset_index()
    highest_scores = innings_scores.groupby('batter')['runs_off_bat'].max().reset_index()
    highest_scores.columns = ['batter', 'highest_score']
    batter_stats = batter_stats.merge(highest_scores, on='batter', how='left')
    batter_stats['highest_score'] = batter_stats['highest_score'].fillna(0).astype(int)
    
    # Sort by runs and get top players
    batter_stats = batter_stats.sort_values('runs_off_bat', ascending=False).head(10)
    
    return batter_stats

def create_player_stats_cards(df, team, phase=None):
    """Create player statistics cards visualization"""
    stats = get_player_statistics(df, team, phase)
    
    if stats.empty:
        return HTML("<p>No player statistics available</p>")
    
    div_id = f"player_stats_{uuid.uuid4().hex[:8]}"
    
    # Prepare data for display
    players_data = []
    for idx, row in stats.iterrows():
        players_data.append({
            'name': str(row['batter']),
            'runs': int(row['runs_off_bat']),
            'balls': int(row['ball']),
            'sr': float(row['strike_rate']),
            'avg': float(row['average']),
            'fours': int(row['fours']),
            'sixes': int(row['sixes']),
            'dismissals': int(row['is_wicket']),
            'highest': int(row['highest_score'])
        })
    
    data_json = json.dumps(players_data)
    
    title_parts = [f"{team} - Player Statistics"]
    if phase:
        title_parts.append(f"({phase})")
    title = " ".join(title_parts)
    
    html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <style>
            body {{ 
                margin: 0; 
                padding: 20px; 
                font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                background: #f5f5f5;
            }}
            .stats-title {{ 
                text-align: center; 
                font-size: 24px; 
                font-weight: bold; 
                margin-bottom: 20px;
                color: #333;
            }}
            .stats-container {{
                display: grid;
                grid-template-columns: repeat(auto-fill, minmax(280px, 1fr));
                gap: 15px;
                max-width: 1400px;
                margin: 0 auto;
            }}
            .player-card {{
                border-radius: 12px;
                padding: 20px;
                color: white;
                box-shadow: 0 4px 15px rgba(0,0,0,0.2);
                transition: transform 0.3s ease, box-shadow 0.3s ease;
                position: relative;
                overflow: hidden;
                border-top: 4px solid;
            }}
            .player-card:hover {{
                transform: translateY(-5px);
                box-shadow: 0 8px 25px rgba(0,0,0,0.3);
            }}
            .player-card::before {{
                content: '';
                position: absolute;
                top: 0;
                left: 0;
                width: 100%;
                height: 100%;
                background: linear-gradient(135deg, rgba(255,255,255,0.1) 0%, rgba(255,255,255,0) 100%);
                pointer-events: none;
            }}
            .player-rank {{
                position: absolute;
                top: 10px;
                right: 10px;
                background: rgba(255,255,255,0.3);
                width: 40px;
                height: 40px;
                border-radius: 50%;
                display: flex;
                align-items: center;
                justify-content: center;
                font-size: 18px;
                font-weight: bold;
                border: 2px solid rgba(255,255,255,0.5);
            }}
            .player-name {{
                font-size: 18px;
                font-weight: bold;
                margin-bottom: 15px;
                padding-right: 50px;
                line-height: 1.3;
            }}
            .main-stats {{
                display: flex;
                justify-content: space-between;
                margin-bottom: 15px;
                padding: 12px;
                background: rgba(255,255,255,0.15);
                border-radius: 8px;
                backdrop-filter: blur(10px);
            }}
            .stat-item {{
                text-align: center;
            }}
            .stat-value {{
                font-size: 24px;
                font-weight: bold;
                display: block;
                margin-bottom: 3px;
            }}
            .stat-label {{
                font-size: 11px;
                opacity: 0.9;
                text-transform: uppercase;
                letter-spacing: 0.5px;
            }}
            .secondary-stats {{
                display: grid;
                grid-template-columns: repeat(3, 1fr);
                gap: 8px;
            }}
            .secondary-stat {{
                background: rgba(255,255,255,0.1);
                padding: 8px;
                border-radius: 6px;
                display: flex;
                justify-content: space-between;
                align-items: center;
            }}
            .secondary-label {{
                font-size: 12px;
                opacity: 0.9;
            }}
            .secondary-value {{
                font-size: 15px;
                font-weight: bold;
            }}
            .boundaries {{
                display: flex;
                gap: 10px;
                margin-top: 10px;
            }}
            .boundary-badge {{
                flex: 1;
                background: rgba(255,255,255,0.2);
                padding: 8px;
                border-radius: 6px;
                text-align: center;
            }}
            .boundary-value {{
                font-size: 20px;
                font-weight: bold;
                display: block;
            }}
            .boundary-label {{
                font-size: 11px;
                opacity: 0.9;
            }}
        </style>
    </head>
    <body>
        <div class="stats-title">{title}</div>
        <div class="stats-container" id="{div_id}"></div>
        
        <script>
        (function() {{
            const players = {data_json};
            const container = document.getElementById('{div_id}');
            
            players.forEach((player, index) => {{
                const card = document.createElement('div');
                card.className = 'player-card';
                
                // Solid professional colors based on rank
                const solidColors = [
                    { bg: '#2563eb', accent: '#1e40af' },  // Blue
                    { bg: '#dc2626', accent: '#991b1b' },  // Red
                    { bg: '#059669', accent: '#047857' },  // Green
                    { bg: '#7c3aed', accent: '#6d28d9' },  // Purple
                    { bg: '#ea580c', accent: '#c2410c' },  // Orange
                    { bg: '#0891b2', accent: '#0e7490' },  // Cyan
                    { bg: '#db2777', accent: '#be185d' },  // Pink
                    { bg: '#65a30d', accent: '#4d7c0f' },  // Lime
                    { bg: '#4f46e5', accent: '#4338ca' },  // Indigo
                    { bg: '#0d9488', accent: '#0f766e' }   // Teal
                ];
                
                const colorScheme = solidColors[index % solidColors.length];
                card.style.background = colorScheme.bg;
                card.style.borderTop = `4px solid ${colorScheme.accent}`;
                card.style.boxShadow = `0 4px 15px ${colorScheme.accent}40`;
                
                card.innerHTML = `
                    <div class="player-rank">${{index + 1}}</div>
                    <div class="player-name">${{player.name}}</div>
                    
                    <div class="main-stats">
                        <div class="stat-item">
                            <span class="stat-value">${{player.runs}}</span>
                            <span class="stat-label">Runs</span>
                        </div>
                        <div class="stat-item">
                            <span class="stat-value">${{player.balls}}</span>
                            <span class="stat-label">Balls</span>
                        </div>
                        <div class="stat-item">
                            <span class="stat-value">${{player.sr.toFixed(1)}}</span>
                            <span class="stat-label">Strike Rate</span>
                        </div>
                    </div>
                    
                    <div class="boundaries">
                        <div class="boundary-badge">
                            <span class="boundary-value">${{player.fours}}</span>
                            <span class="boundary-label">Fours</span>
                        </div>
                        <div class="boundary-badge">
                            <span class="boundary-value">${{player.sixes}}</span>
                            <span class="boundary-label">Sixes</span>
                        </div>
                    </div>
                    
                    <div class="secondary-stats">
                        <div class="secondary-stat">
                            <span class="secondary-label">Average</span>
                            <span class="secondary-value">${{player.avg.toFixed(1)}}</span>
                        </div>
                        <div class="secondary-stat">
                            <span class="secondary-label">Highest</span>
                            <span class="secondary-value">${{player.highest}}</span>
                        </div>
                        <div class="secondary-stat">
                            <span class="secondary-label">Dismissals</span>
                            <span class="secondary-value">${{player.dismissals}}</span>
                        </div>
                    </div>
                `;
                
                container.appendChild(card);
            }});
        }})();
        </script>
    </body>
    </html>
    """
    return HTML(html)

print("✅ Player statistics cards visualization created!")

✅ Player statistics cards visualization created!


In [17]:
def create_bowling_length_map(df, team, phase=None, bowler_type=None):
    """Create 3D bowling length visualization with zones and percentages"""
    pitch_data = generate_pitch_map_data_complete(df, team=team, phase=phase, bowler_type=bowler_type)
    
    if not pitch_data:
        return HTML("<p>No data available for bowling length map</p>")
    
    div_id = f"bowling_length_{uuid.uuid4().hex[:8]}"
    data_json = json.dumps(pitch_data)
    
    title_parts = [f"{team} - Bowling Length Analysis"]
    if phase:
        title_parts.append(f"({phase})")
    if bowler_type:
        title_parts.append(f"vs {bowler_type}")
    title = " ".join(title_parts)
    
    html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/three.js/r128/three.min.js"></script>
        <script src="https://cdn.jsdelivr.net/npm/three@0.128.0/examples/js/controls/OrbitControls.js"></script>
        <style>
            body {{ margin: 0; padding: 20px; font-family: Arial, sans-serif; background: #1a1a1a; }}
            .bowling-container {{ position: relative; text-align: center; }}
            .bowling-title {{ 
                text-align: center; 
                font-size: 24px; 
                font-weight: bold; 
                margin-bottom: 20px;
                color: white;
                text-transform: uppercase;
                letter-spacing: 2px;
            }}
            #{div_id} {{ 
                border: 2px solid #333; 
                border-radius: 8px; 
                display: inline-block;
                box-shadow: 0 8px 30px rgba(0,0,0,0.5);
            }}
            .stats-overlay {{
                position: absolute;
                top: 80px;
                right: 40px;
                background: rgba(0,0,0,0.85);
                padding: 20px;
                border-radius: 10px;
                color: white;
                font-size: 14px;
                min-width: 200px;
                border: 2px solid #444;
            }}
            .zone-stat {{
                margin: 12px 0;
                padding: 10px;
                background: rgba(255,255,255,0.1);
                border-radius: 6px;
                display: flex;
                justify-content: space-between;
                align-items: center;
            }}
            .zone-name {{
                font-weight: bold;
                text-transform: uppercase;
                font-size: 12px;
                letter-spacing: 1px;
            }}
            .zone-percentage {{
                font-size: 24px;
                font-weight: bold;
            }}
            .short {{ color: #ff6b6b; }}
            .length {{ color: #ffd93d; }}
            .full {{ color: #6bcf7f; }}
            .yorker {{ color: #4dabf7; }}
            .legend-title {{
                font-weight: bold;
                margin-bottom: 15px;
                font-size: 16px;
                border-bottom: 2px solid #666;
                padding-bottom: 10px;
            }}
            .view-controls {{
                position: absolute;
                top: 80px;
                left: 20px;
                background: rgba(0,0,0,0.85);
                padding: 15px;
                border-radius: 10px;
                color: white;
                font-size: 12px;
                border: 2px solid #444;
            }}
            .view-btn {{
                background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                border: none;
                color: white;
                padding: 8px 12px;
                margin: 4px 0;
                border-radius: 6px;
                cursor: pointer;
                width: 100%;
                font-weight: bold;
                transition: all 0.3s;
            }}
            .view-btn:hover {{
                transform: translateY(-2px);
                box-shadow: 0 4px 12px rgba(102, 126, 234, 0.4);
            }}
            .controls-title {{
                font-weight: bold;
                margin-bottom: 10px;
                font-size: 14px;
                border-bottom: 2px solid #666;
                padding-bottom: 8px;
            }}
        </style>
    </head>
    <body>
        <div class="bowling-container">
            <div class="bowling-title">{title}</div>
            <div id="{div_id}"></div>
            <div class="view-controls">
                <div class="controls-title">VIEW ANGLES</div>
                <button class="view-btn" onclick="setTopView()">📐 Top View</button>
                <button class="view-btn" onclick="setBowlerView()">🎯 Bowler End</button>
                <button class="view-btn" onclick="setBatterView()">🏏 Batter End</button>
                <button class="view-btn" onclick="setSideView()">👁️ Side View</button>
                <button class="view-btn" onclick="resetView()">🔄 Reset</button>
            </div>
            <div class="stats-overlay">
                <div class="legend-title">BOWLING LENGTH %</div>
                <div id="zone-stats"></div>
            </div>
        </div>
        
        <script>
        (function() {{
            const pitchData = {data_json};
            const scene = new THREE.Scene();
            scene.background = new THREE.Color(0x1a1a1a);
            scene.fog = new THREE.Fog(0x1a1a1a, 30, 60);
            
            const camera = new THREE.PerspectiveCamera(45, 900/700, 0.1, 1000);
            camera.position.set(5, 18, 25);
            camera.lookAt(0, 0, 11);
            
            const renderer = new THREE.WebGLRenderer({{ antialias: true, alpha: true }});
            renderer.setSize(900, 700);
            renderer.shadowMap.enabled = true;
            renderer.shadowMap.type = THREE.PCFSoftShadowMap;
            document.getElementById('{div_id}').appendChild(renderer.domElement);
            
            const controls = new THREE.OrbitControls(camera, renderer.domElement);
            controls.enableDamping = true;
            controls.dampingFactor = 0.05;
            controls.target.set(0, 0, 11);
            controls.maxPolarAngle = Math.PI / 2.2;
            
            // Lighting
            const ambientLight = new THREE.AmbientLight(0xffffff, 0.4);
            scene.add(ambientLight);
            
            const mainLight = new THREE.DirectionalLight(0xffffff, 0.8);
            mainLight.position.set(10, 20, 10);
            mainLight.castShadow = true;
            mainLight.shadow.mapSize.width = 2048;
            mainLight.shadow.mapSize.height = 2048;
            scene.add(mainLight);
            
            const fillLight = new THREE.DirectionalLight(0xffffff, 0.3);
            fillLight.position.set(-10, 10, -10);
            scene.add(fillLight);
            
            // Ground (dark grass)
            const groundGeometry = new THREE.PlaneGeometry(40, 50);
            const groundMaterial = new THREE.MeshStandardMaterial({{ 
                color: 0x1a4d2e,
                roughness: 0.9
            }});
            const ground = new THREE.Mesh(groundGeometry, groundMaterial);
            ground.rotation.x = -Math.PI / 2;
            ground.position.y = -0.05;
            ground.receiveShadow = true;
            scene.add(ground);
            
            // Define length zones with 3D appearance
            const zones = [
                {{ name: 'YORKER', start: 20, end: 22, color: 0x4dabf7, label: 'YORKER', yPos: 21 }},
                {{ name: 'FULL', start: 16, end: 20, color: 0x6bcf7f, label: 'FULL', yPos: 18 }},
                {{ name: 'LENGTH', start: 10, end: 16, color: 0xffd93d, label: 'LENGTH', yPos: 13 }},
                {{ name: 'SHORT', start: 4, end: 10, color: 0xff6b6b, label: 'SHORT', yPos: 7 }}
            ];
            
            // Create 3D zone blocks with beveled edges
            zones.forEach(zone => {{
                const height = zone.end - zone.start;
                const geometry = new THREE.BoxGeometry(2.8, 0.15, height);
                const material = new THREE.MeshStandardMaterial({{ 
                    color: zone.color,
                    transparent: true,
                    opacity: 0.7,
                    roughness: 0.5,
                    metalness: 0.2
                }});
                const zoneMesh = new THREE.Mesh(geometry, material);
                zoneMesh.position.set(0, 0.075, zone.start + height/2);
                zoneMesh.receiveShadow = true;
                zoneMesh.castShadow = true;
                scene.add(zoneMesh);
                
                // Zone labels (3D text appearance)
                const canvas = document.createElement('canvas');
                const context = canvas.getContext('2d');
                canvas.width = 256;
                canvas.height = 128;
                context.fillStyle = 'white';
                context.font = 'bold 48px Arial';
                context.textAlign = 'center';
                context.fillText(zone.label, 128, 80);
                
                const texture = new THREE.CanvasTexture(canvas);
                const spriteMaterial = new THREE.SpriteMaterial({{ 
                    map: texture,
                    transparent: true,
                    opacity: 0.9
                }});
                const sprite = new THREE.Sprite(spriteMaterial);
                sprite.position.set(-2.5, 0.5, zone.yPos);
                sprite.scale.set(2, 1, 1);
                scene.add(sprite);
            }});
            
            // Create stumps (3D cylindrical)
            const stumpMaterial = new THREE.MeshStandardMaterial({{ 
                color: 0x8B4513,
                roughness: 0.8
            }});
            
            // Bowler's end stumps
            for (let i = -0.15; i <= 0.15; i += 0.15) {{
                const stumpGeometry = new THREE.CylinderGeometry(0.025, 0.025, 0.8, 8);
                const stump = new THREE.Mesh(stumpGeometry, stumpMaterial);
                stump.position.set(i, 0.4, 0);
                stump.castShadow = true;
                scene.add(stump);
            }}
            
            // Batter's end stumps
            for (let i = -0.15; i <= 0.15; i += 0.15) {{
                const stumpGeometry = new THREE.CylinderGeometry(0.025, 0.025, 0.8, 8);
                const stump = new THREE.Mesh(stumpGeometry, stumpMaterial);
                stump.position.set(i, 0.4, 22);
                stump.castShadow = true;
                scene.add(stump);
            }}
            
            // Bails
            const bailMaterial = new THREE.MeshStandardMaterial({{ color: 0x8B4513 }});
            for (let pos of [0, 22]) {{
                for (let i = -0.075; i <= 0.075; i += 0.15) {{
                    const bailGeometry = new THREE.CylinderGeometry(0.01, 0.01, 0.15, 8);
                    const bail = new THREE.Mesh(bailGeometry, bailMaterial);
                    bail.rotation.z = Math.PI / 2;
                    bail.position.set(i, 0.8, pos);
                    bail.castShadow = true;
                    scene.add(bail);
                }}
            }}
            
            // Pitch center line (purple)
            const lineGeometry = new THREE.BoxGeometry(0.15, 0.02, 22);
            const lineMaterial = new THREE.MeshStandardMaterial({{ color: 0x9b59b6 }});
            const centerLine = new THREE.Mesh(lineGeometry, lineMaterial);
            centerLine.position.set(0, 0.16, 11);
            scene.add(centerLine);
            
            // Crease lines (white)
            const creaseMaterial = new THREE.MeshStandardMaterial({{ color: 0xffffff }});
            for (let z of [0, 22]) {{
                const creaseGeometry = new THREE.BoxGeometry(2.8, 0.02, 0.1);
                const crease = new THREE.Mesh(creaseGeometry, creaseMaterial);
                crease.position.set(0, 0.16, z);
                scene.add(crease);
            }}
            
            // Create balls with shadows
            const ballMeshes = [];
            const colorMap = {{
                'red': 0xff0000,
                'purple': 0x9c27b0,
                'green': 0x00ff00,
                'blue': 0x2196f3,
                'gray': 0x808080
            }};
            
            pitchData.forEach(ball => {{
                const radius = ball.size * 0.04;
                const ballGeometry = new THREE.SphereGeometry(radius, 16, 16);
                const ballMaterial = new THREE.MeshStandardMaterial({{ 
                    color: colorMap[ball.color],
                    roughness: 0.6,
                    metalness: 0.3
                }});
                const sphere = new THREE.Mesh(ballGeometry, ballMaterial);
                sphere.position.set(ball.x, radius + 0.15, ball.y);
                sphere.castShadow = true;
                sphere.userData = {{
                    runs: ball.runs,
                    wicket: ball.wicket,
                    batter: ball.batter,
                    bowler: ball.bowler
                }};
                scene.add(sphere);
                ballMeshes.push(sphere);
            }});
            
            // Mouse interaction
            const raycaster = new THREE.Raycaster();
            const mouse = new THREE.Vector2();
            let tooltip = null;
            
            renderer.domElement.addEventListener('mousemove', (event) => {{
                const rect = renderer.domElement.getBoundingClientRect();
                mouse.x = ((event.clientX - rect.left) / rect.width) * 2 - 1;
                mouse.y = -((event.clientY - rect.top) / rect.height) * 2 + 1;
                
                raycaster.setFromCamera(mouse, camera);
                const intersects = raycaster.intersectObjects(ballMeshes);
                
                if (intersects.length > 0) {{
                    const data = intersects[0].object.userData;
                    if (!tooltip) {{
                        tooltip = document.createElement('div');
                        tooltip.style.position = 'absolute';
                        tooltip.style.background = 'rgba(0,0,0,0.9)';
                        tooltip.style.color = 'white';
                        tooltip.style.padding = '10px 15px';
                        tooltip.style.borderRadius = '6px';
                        tooltip.style.fontSize = '12px';
                        tooltip.style.pointerEvents = 'none';
                        tooltip.style.zIndex = '10000';
                        tooltip.style.border = '2px solid #444';
                        document.body.appendChild(tooltip);
                    }}
                    const wicketText = data.wicket ? ' - WICKET!' : '';
                    tooltip.innerHTML = `
                        <strong style="font-size: 14px;">${{data.runs}} run(s)${{wicketText}}</strong><br>
                        <span style="color: #aaa;">Batter:</span> ${{data.batter}}<br>
                        <span style="color: #aaa;">Bowler:</span> ${{data.bowler}}
                    `;
                    tooltip.style.display = 'block';
                    tooltip.style.left = event.clientX + 15 + 'px';
                    tooltip.style.top = event.clientY + 15 + 'px';
                }} else if (tooltip) {{
                    tooltip.style.display = 'none';
                }}
            }});
            
            // Calculate zone percentages
            const total = pitchData.length;
            const zoneCounts = {{
                'YORKER': pitchData.filter(d => d.y >= 20 && d.y <= 22).length,
                'FULL': pitchData.filter(d => d.y >= 16 && d.y < 20).length,
                'LENGTH': pitchData.filter(d => d.y >= 10 && d.y < 16).length,
                'SHORT': pitchData.filter(d => d.y >= 4 && d.y < 10).length
            }};
            
            const statsHtml = [
                {{ name: 'YORKER', count: zoneCounts.YORKER, class: 'yorker' }},
                {{ name: 'FULL', count: zoneCounts.FULL, class: 'full' }},
                {{ name: 'LENGTH', count: zoneCounts.LENGTH, class: 'length' }},
                {{ name: 'SHORT', count: zoneCounts.SHORT, class: 'short' }}
            ].map(stat => {{
                const percentage = total > 0 ? ((stat.count / total) * 100).toFixed(0) : 0;
                return `
                    <div class="zone-stat">
                        <span class="zone-name ${{stat.class}}">${{stat.name}}</span>
                        <span class="zone-percentage ${{stat.class}}">${{percentage}}%</span>
                    </div>
                `;
            }}).join('');
            
            document.getElementById('zone-stats').innerHTML = statsHtml;
            
            // Camera animation helper
            function animateCamera(targetPos, targetLookAt, duration = 1000) {{
                const startPos = camera.position.clone();
                const startTarget = controls.target.clone();
                const startTime = Date.now();
                
                function animate() {{
                    const elapsed = Date.now() - startTime;
                    const progress = Math.min(elapsed / duration, 1);
                    const eased = 1 - Math.pow(1 - progress, 3);
                    
                    camera.position.lerpVectors(startPos, targetPos, eased);
                    controls.target.lerpVectors(startTarget, targetLookAt, eased);
                    controls.update();
                    
                    if (progress < 1) {{
                        requestAnimationFrame(animate);
                    }}
                }}
                animate();
            }}
            
            // View angle functions
            window.setTopView = function() {{
                animateCamera(
                    new THREE.Vector3(0, 35, 11),
                    new THREE.Vector3(0, 0, 11)
                );
            }};
            
            window.setBowlerView = function() {{
                animateCamera(
                    new THREE.Vector3(0, 12, -8),
                    new THREE.Vector3(0, 0, 11)
                );
            }};
            
            window.setBatterView = function() {{
                animateCamera(
                    new THREE.Vector3(0, 12, 30),
                    new THREE.Vector3(0, 0, 11)
                );
            }};
            
            window.setSideView = function() {{
                animateCamera(
                    new THREE.Vector3(25, 15, 11),
                    new THREE.Vector3(0, 0, 11)
                );
            }};
            
            window.resetView = function() {{
                animateCamera(
                    new THREE.Vector3(5, 18, 25),
                    new THREE.Vector3(0, 0, 11)
                );
            }};
            
            // Animation loop
            function animate() {{
                requestAnimationFrame(animate);
                controls.update();
                renderer.render(scene, camera);
            }}
            animate();
        }})();
        </script>
    </body>
    </html>
    """
    return HTML(html)

print("✅ 3D Bowling length visualization with zones created!")

✅ 3D Bowling length visualization with zones created!


## 6. Interactive Dashboard

Create an interactive dashboard with widgets to select teams and generate analysis

In [18]:
# Import required libraries for the dashboard
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML

teams = sorted(df['batting_team'].unique())

# Create widgets
team1_dropdown = widgets.Dropdown(
    options=teams,
    value=teams[0] if len(teams) > 0 else None,
    description='Team 1:',
    style={'description_width': 'initial'}
)

team2_dropdown = widgets.Dropdown(
    options=teams,
    value=teams[1] if len(teams) > 1 else teams[0],
    description='Team 2:',
    style={'description_width': 'initial'}
)

bowler_type_dropdown = widgets.Dropdown(
    options=['Right-Arm Pace', 'Left-Arm Pace', 'Right-Arm Leg Spin', 'Right-Arm Off Spin', 'Left-Arm Orthodox', 'Left-Arm Wrist Spin'],
    value='Right-Arm Pace',
    description='Bowler Type:',
    style={'description_width': 'initial'}
)

phase_dropdown = widgets.Dropdown(
    options=['All Phases', 'Powerplay (1-6)', 'Middle (7-15)', 'Death (16-20)'],
    value='All Phases',
    description='Phase:',
    style={'description_width': 'initial'}
)

generate_button = widgets.Button(
    description='Generate Dashboard',
    button_style='success',
    icon='chart-bar'
)

output = widgets.Output()

def generate_dashboard(b):
    """Generate all visualizations"""
    with output:
        output.clear_output()
        
        print("🏏 IPL TEAM PERFORMANCE DASHBOARD 🏏")
        print("=" * 60)
        
        team1 = team1_dropdown.value
        team2 = team2_dropdown.value
        bowler_type = bowler_type_dropdown.value
        phase = phase_dropdown.value
        phase_filter = None if phase == 'All Phases' else phase
        
        # Phase Comparison
        print(f"\n📊 Phase Analysis: {team1} vs {team2}")
        html1 = plot_phase_comparison(df, team1, team2)
        display(html1)
        
        # Advanced Pitch Visualizations (4-panel view)
        print(f"\n🎯 Advanced Pitch Analysis - 4 Views")
        print(f"Wickets, Hitting, Heat Map, and Combined Analysis")
        
        # Team 1 Advanced Pitch Analysis
        print(f"\n📍 {team1} - Multi-Panel Pitch Analysis")
        advanced_pitch1 = create_advanced_pitch_viz(df, team1, phase=phase_filter, bowler_type=bowler_type)
        display(advanced_pitch1)
        
        # Team 2 Advanced Pitch Analysis
        print(f"\n📍 {team2} - Multi-Panel Pitch Analysis")
        advanced_pitch2 = create_advanced_pitch_viz(df, team2, phase=phase_filter, bowler_type=bowler_type)
        display(advanced_pitch2)
        
        # Pitch Maps
        print(f"\n🎯 Pitch Map Analysis")
        print(f"Showing ball landing positions and outcomes")
        
        # Team 1 Pitch Map
        print(f"\n📍 {team1} Pitch Map")
        pitch1 = create_pitch_map(df, team1, phase=phase_filter, bowler_type=bowler_type)
        display(pitch1)
        
        # Team 2 Pitch Map
        print(f"\n📍 {team2} Pitch Map")
        pitch2 = create_pitch_map(df, team2, phase=phase_filter, bowler_type=bowler_type)
        display(pitch2)
        
        # Stumps View
        print(f"\n🎯 Stumps View Analysis")
        print(f"Behind the bowler perspective showing line and length")
        
        print(f"\n📍 {team1} Stumps View")
        stumps1 = create_stumps_view(df, team1, phase=phase_filter)
        display(stumps1)
        
        print(f"\n📍 {team2} Stumps View")
        stumps2 = create_stumps_view(df, team2, phase=phase_filter)
        display(stumps2)
        
        # Wagon Wheel (Ground Map)
        print(f"\n🎯 Wagon Wheel (Ground Map)")
        print(f"Shot directions and scoring zones")
        
        print(f"\n📍 {team1} Wagon Wheel")
        wagon1 = create_wagon_wheel(df, team1, phase=phase_filter)
        display(wagon1)
        
        print(f"\n📍 {team2} Wagon Wheel")
        wagon2 = create_wagon_wheel(df, team2, phase=phase_filter)
        display(wagon2)
        
        # Player Statistics Cards
        print(f"\n📊 Player Statistics - Top Performers")
        print(f"Comprehensive batting statistics and metrics")
        
        print(f"\n📍 {team1} - Top Batters")
        player_stats1 = create_player_stats_cards(df, team1, phase=phase_filter)
        display(player_stats1)
        
        print(f"\n📍 {team2} - Top Batters")
        player_stats2 = create_player_stats_cards(df, team2, phase=phase_filter)
        display(player_stats2)
        
        # Bowling Length Analysis
        print(f"\n🎯 Bowling Length Analysis")
        print(f"3D visualization with color-coded zones and percentages")
        
        print(f"\n📍 {team1} - Bowling Length Distribution")
        bowling1 = create_bowling_length_map(df, team1, phase=phase_filter, bowler_type=bowler_type)
        display(bowling1)
        
        print(f"\n📍 {team2} - Bowling Length Distribution")
        bowling2 = create_bowling_length_map(df, team2, phase=phase_filter, bowler_type=bowler_type)
        display(bowling2)
        
        # Matchup Analysis - Team 1
        print(f"\n🎯 Player Matchups - {team1} vs {bowler_type}")
        html2 = plot_matchup_heatmap(df, team1, bowler_type)
        display(html2)
        
        # Matchup Analysis - Team 2
        print(f"\n🎯 Player Matchups - {team2} vs {bowler_type}")
        html3 = plot_matchup_heatmap(df, team2, bowler_type)
        display(html3)
        
        # Runs Distribution
        print(f"\n📈 Runs Distribution Comparison")
        html4 = plot_runs_distribution(df, team1)
        display(html4)
        
        html5 = plot_runs_distribution(df, team2)
        display(html5)
        
        print("\n✅ Dashboard generated successfully!")

generate_button.on_click(generate_dashboard)

# Display dashboard controls
display(HTML("<h2>🏏 IPL Team Performance Dashboard</h2>"))
display(HTML("<p>Select two teams to compare their performance across different phases and matchups.</p>"))
display(HTML("<p><strong>Professional Cricket Visualizations:</strong></p>"))
display(HTML("""
<ul style='margin: 10px 0; line-height: 1.8;'>
    <li>🎯 <strong>Advanced Pitch Maps</strong>: 4-panel analysis
        <ul style='margin-left: 20px; font-size: 0.9em;'>
            <li>Wickets scatter plot</li>
            <li>Hitting zones (non-wicket deliveries)</li>
            <li>Density heat map with contours</li>
            <li>Combined overlay view</li>
        </ul>
    </li>
    <li>📊 <strong>Stumps View</strong>: Behind-the-bowler perspective (line & length)</li>
    <li>⚾ <strong>Wagon Wheel</strong>: Shot directions & scoring zones (ground map)</li>
    <li>📈 <strong>Player Stats</strong>: Batter vs bowler matchups</li>
</ul>
"""))
display(widgets.VBox([team1_dropdown, team2_dropdown, bowler_type_dropdown, phase_dropdown, generate_button, output]))

print("\n" + "="*60)
print("✅ Dashboard is ready! Select teams and click 'Generate Dashboard'")
print("="*60)


✅ Dashboard is ready! Select teams and click 'Generate Dashboard'
